In [1]:
// See https://jonnylaw.rocks/posts/2019-04-15-scala-and-jupyter-notebook-with-almond/ for some info about setup.

import fastparse._

import fastparse._

In [2]:
import $ivy.`com.beachape::enumeratum:1.7.3`
import enumeratum.{Enum, EnumEntry}


import $ivy.$
import enumeratum.{Enum, EnumEntry}

In [3]:
// Define the set of built-in symbols supported in µDhall.

sealed abstract class Constant(override val entryName: String) extends EnumEntry {}

object Constant extends Enum[Constant] {
  override def values = findValues

  case object Natural         extends Constant("Natural")
  case object NaturalFold     extends Constant("Natural/fold")
  case object NaturalIsZero   extends Constant("Natural/isZero")
  case object NaturalSubtract extends Constant("Natural/subtract")
  case object Kind            extends Constant("Kind")
  case object Type            extends Constant("Type")
}

defined class Constant
defined object Constant

In [4]:
// Define the set of built-in binary operators supported in µDhall.

sealed abstract class Operator(override val entryName: String, prec: Int) extends EnumEntry

object Operator extends Enum[Operator] {
  val values = findValues
  // These operators work only with values of type Natural.
  case object Plus extends Operator("+", 10)
  case object Times extends Operator("*", 20)
}

defined class Operator
defined object Operator

In [5]:
sealed trait Expr

object Expr {
  // Natural literals, for example 123
  final case class NaturalLiteral(value: Int) extends Expr {
      require(value >= 0)
  }
  // Variables with their de Bruijn indices.
  final case class Variable(name: String, dbi: Int = 0) extends Expr {
      require (dbi >= 0)
  }
  // λ(name : tpe) → body  -- Function literal value.
  final case class Lambda(name: String, tpe: Expr, body: Expr) extends Expr

  // ∀(name : tpe) → body  -- Function type.
  final case class Forall(name: String, tpe: Expr, body: Expr) extends Expr

  // let name = subst in body  -- Locally scoped variable definition.
  final case class Let(name: String, subst: Expr, body: Expr) extends Expr

  // body : tpe   -- Type annotation.
  final case class Annotated(body: Expr, tpe: Expr) extends Expr

  // func arg   -- Application of a function to an argument.
  final case class Applied(func: Expr, arg: Expr) extends Expr

  // Built-in constant symbols such as "Natural" or "Type".
  final case class Builtin(constant: Constant) extends Expr

  // Binary operations such as "n + 123".
  final case class BinaryOp(l: Expr, op: Operator, r: Expr) extends Expr
}

cmd5.sc:179: The outer reference in this type test cannot be checked at run time.
  final case class NaturalLiteral(value: Int) extends Expr {
                   ^
cmd5.sc:183: The outer reference in this type test cannot be checked at run time.
  final case class Variable(name: String, dbi: Int = 0) extends Expr {
                   ^
cmd5.sc:187: The outer reference in this type test cannot be checked at run time.
  final case class Lambda(name: String, tpe: Expr, body: Expr) extends Expr
                   ^
cmd5.sc:190: The outer reference in this type test cannot be checked at run time.
  final case class Forall(name: String, tpe: Expr, body: Expr) extends Expr
                   ^
cmd5.sc:193: The outer reference in this type test cannot be checked at run time.
  final case class Let(name: String, subst: Expr, body: Expr) extends Expr
                   ^
cmd5.sc:196: The outer reference in this type test cannot be checked at run time.
  final case class Annotated(body: Expr, tpe

defined trait Expr
defined object Expr

In [6]:
val test = Expr.BinaryOp(Expr.NaturalLiteral(123), Operator.Plus, Expr.Variable("a", 0))

test: Expr.BinaryOp = BinaryOp(
  l = NaturalLiteral(value = 123),
  op = Plus,
  r = Variable(name = "a", dbi = 0)
)

In [7]:
object DSL { // Helper methods for creating µDhall values more easily in Scala.
    import Expr._
    import Constant._
    import Operator._
    
    implicit class IntroduceVar(name: String) {
        def ! : Variable = Variable(name)
    }
    implicit class IntroduceNatural(n: Int) {
        def  ! : NaturalLiteral = NaturalLiteral(n)
    }
    implicit class IntroduceSymbol(c: Constant) {
        def ! : Expr = Builtin(c)
    }
    implicit class NaturalOps(x: Expr) {
        def +(other: Expr) : Expr = BinaryOp(x, Plus, other)
        def *(other: Expr) : Expr = BinaryOp(x, Times, other)
    }

}

defined object DSL

In [9]:
{
    //import scala.language.postfixOps
    import DSL._
    import Constant._
    
    val test = ("n".! + 123.! ,  Type!)
}

import DSL._
import Constant._
test: (Expr, Expr) = (
  BinaryOp(
    l = Variable(name = "n", dbi = 0),
    op = Plus,
    r = NaturalLiteral(value = 123)
  ),
  Builtin(constant = Type)
)